In [ ]:
# | default_exp confluent_kafka

In [ ]:
# | export

from typing import List, Dict, Any, Optional, Callable, Tuple, Generator
from os import environ
import string
from contextlib import contextmanager

import asyncio
from asyncio import BaseEventLoop
from inspect import iscoroutinefunction

import numpy as np
import confluent_kafka
from confluent_kafka import KafkaException, Consumer, Producer, Message, KafkaError
from confluent_kafka.admin import AdminClient, NewTopic
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import time
from threading import Thread
from asyncer import syncify

import fast_kafka_api.logger

fast_kafka_api.logger.should_supress_timestamps = True

from fast_kafka_api.logger import get_logger

In [ ]:
# | export
logger = get_logger(__name__)

In [ ]:
logger = get_logger(__name__, level=0)
logger.debug("ok")

[DEBUG] __main__: ok


In [ ]:
import time

import pytest

import nest_asyncio
from rich.pretty import pprint

# from fast_kafka_api.kafka.service.servers import create_missing_topics

In [ ]:
# | eval: false

# allows async calls in notebooks

nest_asyncio.apply()

In [ ]:
# | export


def create_missing_topics(
    admin: AdminClient,
    topic_names: List[str],
    *,
    num_partitions: Optional[int] = None,
    replication_factor: Optional[int] = None,
    **kwargs,
) -> None:
    if not replication_factor:
        replication_factor = len(admin.list_topics().brokers)
    if not num_partitions:
        num_partitions = replication_factor
    existing_topics = list(admin.list_topics().topics.keys())
    logger.debug(
        f"create_missing_topics({topic_names}): existing_topics={existing_topics}, num_partitions={num_partitions}, replication_factor={replication_factor}"
    )
    new_topics = [
        NewTopic(
            topic,
            num_partitions=num_partitions,
            replication_factor=replication_factor,
            **kwargs,
        )
        for topic in topic_names
        if topic not in existing_topics
    ]
    if len(new_topics):
        logger.info(f"create_missing_topics({topic_names}): new_topics = {new_topics}")
        #         nlsep = "\n - "
        #         logger.info(f"create_missing_topics({topic_names}): creating topics:{nlsep}{nlsep.join([str(t) for t in new_topics])}")
        fs = admin.create_topics(new_topics)
        results = {k: f.result() for k, f in fs.items()}
        time.sleep(2)

In [ ]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
    "bootstrap.servers": f"{kafka_server_url}:{kafka_server_port}",
    "group.id": f"{kafka_server_url}:{kafka_server_port}_group",  # ToDo: Figure out msg deletion from kafka after consuming once
    "auto.offset.reset": "earliest",
}

kafka_admin = AdminClient(kafka_config)

create_missing_topics(kafka_admin, ["A", "B", "C"])
existing_topics = kafka_admin.list_topics().topics.keys()
print("Existing topics:\n - " + "\n - ".join(sorted(existing_topics)))
assert set(["A", "B", "C"]) <= existing_topics

[DEBUG] __main__: create_missing_topics(['A', 'B', 'C']): existing_topics=['my_topic_error', 'my_test_topic', 'my_topic_1', 'training_status', 'prediction_request', '__consumer_offsets', 'B', 'C', 'my_topic_3', 'training_request', 'my_topic_4', 'my_topic_2', 'my_topic_test', 'prediction_status'], num_partitions=3, replication_factor=3
[INFO] __main__: create_missing_topics(['A', 'B', 'C']): new_topics = [NewTopic(topic=A,num_partitions=3)]


%4|1671618705.901|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1671618705.901|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


Existing topics:
 - A
 - B
 - C
 - __consumer_offsets
 - my_test_topic
 - my_topic_1
 - my_topic_2
 - my_topic_3
 - my_topic_4
 - my_topic_error
 - my_topic_test
 - prediction_request
 - prediction_status
 - training_request
 - training_status


In [ ]:
fs = kafka_admin.delete_topics(topics=["A"])
results = {k: f.result() for k, f in fs.items()}
existing_topics = kafka_admin.list_topics().topics.keys()
existing_topics = kafka_admin.list_topics().topics.keys()
print("Existing topics:\n - " + "\n - ".join(sorted(existing_topics)))
assert "A" not in existing_topics

Existing topics:
 - B
 - C
 - __consumer_offsets
 - my_test_topic
 - my_topic_1
 - my_topic_2
 - my_topic_3
 - my_topic_4
 - my_topic_error
 - my_topic_test
 - prediction_request
 - prediction_status
 - training_request
 - training_status


In [ ]:
# | export


def _consume_all_messages(c: Consumer, timeout=0.1, no_retries: int = 25):
    while True:
        for i in range(no_retries):
            msg = c.poll(timeout=timeout)
            if msg:
                if msg.error():
                    logger.warning(f"Error while consuming message: {msg.error()}")
                break
        break


@contextmanager
def create_testing_topic(
    kafka_config: Dict[str, Any], topic_prefix: str, seed: int
) -> Generator[Tuple[str, Consumer, Producer], None, None]:
    # create random topic name
    rng = np.random.default_rng(seed)
    topic = topic_prefix + "".join(rng.choice(list(string.ascii_lowercase), size=10))

    # delete topic if it already exists
    admin = AdminClient(kafka_config)
    existing_topics = admin.list_topics().topics.keys()
    if topic in existing_topics:
        logger.warning(f"topic {topic} exists, deleting it...")
        fs = admin.delete_topics(topics=[topic])
        results = {k: f.result() for k, f in fs.items()}
        time.sleep(1)

    try:
        # create topic if needed
        create_missing_topics(admin, [topic])

        # create consumer and producer for the topic
        c = Consumer(kafka_config)
        c.subscribe([topic])
        p = Producer(kafka_config)

        yield topic, c, p

    finally:
        pass
        # cleanup if needed again
        #         _consume_all_messages(c)
        fs = admin.delete_topics(topics=[topic])
        results = {k: f.result() for k, f in fs.items()}
        time.sleep(1)

In [ ]:
for _ in range(3):
    print("*" * 120)
    with create_testing_topic(kafka_config, "my_topic_", 723453) as (topic, c, p):
        print(f"{topic}")

    existing_topics = kafka_admin.list_topics().topics.keys()
    assert topic not in existing_topics

************************************************************************************************************************
[DEBUG] __main__: create_missing_topics(['my_topic_sbvqouvlnt']): existing_topics=['my_topic_error', 'my_test_topic', 'my_topic_1', 'training_status', 'prediction_request', '__consumer_offsets', 'B', 'C', 'my_topic_3', 'training_request', 'my_topic_4', 'my_topic_2', 'my_topic_test', 'prediction_status'], num_partitions=3, replication_factor=3
[INFO] __main__: create_missing_topics(['my_topic_sbvqouvlnt']): new_topics = [NewTopic(topic=my_topic_sbvqouvlnt,num_partitions=3)]


%4|1671618707.071|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1671618707.071|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


my_topic_sbvqouvlnt


%4|1671618708.211|CONFWARN|rdkafka#producer-4| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1671618708.211|CONFWARN|rdkafka#producer-4| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


************************************************************************************************************************
[DEBUG] __main__: create_missing_topics(['my_topic_sbvqouvlnt']): existing_topics=['my_topic_error', 'my_test_topic', 'my_topic_1', 'training_status', 'prediction_request', '__consumer_offsets', 'B', 'C', 'my_topic_3', 'training_request', 'my_topic_4', 'my_topic_2', 'my_topic_test', 'prediction_status'], num_partitions=3, replication_factor=3
[INFO] __main__: create_missing_topics(['my_topic_sbvqouvlnt']): new_topics = [NewTopic(topic=my_topic_sbvqouvlnt,num_partitions=3)]


%4|1671618709.253|CONFWARN|rdkafka#producer-5| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1671618709.253|CONFWARN|rdkafka#producer-5| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
%4|1671618710.290|CONFWARN|rdkafka#producer-7| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1671618710.290|CONFWARN|rdkafka#producer-7| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


my_topic_sbvqouvlnt
************************************************************************************************************************
[DEBUG] __main__: create_missing_topics(['my_topic_sbvqouvlnt']): existing_topics=['my_topic_error', 'my_test_topic', 'my_topic_1', 'training_status', 'prediction_request', '__consumer_offsets', 'B', 'C', 'my_topic_3', 'training_request', 'my_topic_4', 'my_topic_2', 'my_topic_test', 'prediction_status'], num_partitions=3, replication_factor=3
[INFO] __main__: create_missing_topics(['my_topic_sbvqouvlnt']): new_topics = [NewTopic(topic=my_topic_sbvqouvlnt,num_partitions=3)]


%4|1671618711.333|CONFWARN|rdkafka#producer-8| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1671618711.333|CONFWARN|rdkafka#producer-8| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
%4|1671618712.421|CONFWARN|rdkafka#producer-10| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1671618712.421|CONFWARN|rdkafka#producer-10| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


my_topic_sbvqouvlnt


In [ ]:
# | export


class AIOProducer:
    """Async producer

    Adapted companion code of the blog post "Integrating Kafka With Python Asyncio Web Applications"
    https://www.confluent.io/blog/kafka-python-asyncio-integration/

    https://github.com/confluentinc/confluent-kafka-python/blob/master/examples/asyncio_example.py

    """

    def __init__(
        self, config: Dict[str, Any], loop: Optional[BaseEventLoop] = None
    ) -> None:
        self._loop = loop or asyncio.get_event_loop()
        self._producer = Producer(config)
        self._cancelled = False
        self._poll_thread = Thread(target=self._poll_loop)
        self._poll_thread.start()

    def _poll_loop(self) -> None:
        while not self._cancelled:
            self._producer.poll(0.1)

    def close(self) -> None:
        """Shutdowns the pooling thread pool"""
        self._cancelled = True
        self._poll_thread.join()

    def produce(
        self,
        topic: str,
        value: bytes,
        on_delivery: Optional[Callable[[KafkaError, Message], Any]] = None,
    ) -> "asyncio.Future[Any]":
        """An awaitable produce method

        Params:
            topic: name of the topic
            value: encoded message
            on_delivery: callback function to be called on delivery from a separate thread

        Returns:
            Awaitable future

        Raises:
            ValueError: if a coroutine passed as on_delivery
        """
        if on_delivery and iscoroutinefunction(on_delivery):
            raise ValueError("can only call synchronous code for now")

        result = self._loop.create_future()

        def ack(
            err: KafkaError,
            msg: Message,
            self: "AIOProducer" = self,
            result: "asyncio.Future[Any]" = result,
            on_delivery: Optional[Callable[[KafkaError, Message], Any]] = on_delivery,
        ) -> None:
            if err:
                self._loop.call_soon_threadsafe(
                    result.set_exception, KafkaException(err)
                )
            else:
                self._loop.call_soon_threadsafe(result.set_result, msg)

            if on_delivery:
                self._loop.call_soon_threadsafe(on_delivery, err, msg)

        self._producer.produce(topic, value, on_delivery=ack)

        return result

In [ ]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

config = {
    "bootstrap.servers": f"{kafka_server_url}:{kafka_server_port}",
    "group.id": f"{kafka_server_url}:{kafka_server_port}_group",  # ToDo: Figure out msg deletion from kafka after consuming once
    "auto.offset.reset": "earliest",
}

# kafka_admin = AdminClient(config)
# create_missing_topics(admin=kafka_admin, topic_names=["test_topic_aioproducer"])

In [ ]:
def on_delivery(*args, **kwargs):
    logger.info(f"Hello world!, args={args}, kwargs={kwargs}")
    time.sleep(1)
    logger.info(f"Goodbye world!, args={args}, kwargs={kwargs}")


async def async_on_delivery(*args, **kwargs):
    logger.info(f"Async hello world!, args={args}, kwargs={kwargs}")


async def test_me(topic, config=config):
    aioproducer = AIOProducer(config)

    fx = [
        aioproducer.produce(topic, "hello", on_delivery=on_delivery) for _ in range(5)
    ]

    with pytest.raises(ValueError) as e:
        aioproducer.produce(topic, "hello", on_delivery=async_on_delivery)
    assert str(e.value) == "can only call synchronous code for now"

    # await fx
    pprint([await f for f in fx])

    aioproducer.close()


with create_testing_topic(kafka_config, "test_topic_aioproducer_", 0) as (topic, c, p):
    asyncio.run(test_me(topic))

[DEBUG] __main__: create_missing_topics(['test_topic_aioproducer_wqnhibbaev']): existing_topics=['my_topic_error', 'my_test_topic', 'my_topic_1', 'training_status', 'prediction_request', '__consumer_offsets', 'B', 'C', 'my_topic_3', 'training_request', 'my_topic_4', 'my_topic_2', 'my_topic_test', 'prediction_status'], num_partitions=3, replication_factor=3
[INFO] __main__: create_missing_topics(['test_topic_aioproducer_wqnhibbaev']): new_topics = [NewTopic(topic=test_topic_aioproducer_wqnhibbaev,num_partitions=3)]


%4|1671618713.483|CONFWARN|rdkafka#producer-11| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1671618713.483|CONFWARN|rdkafka#producer-11| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


[INFO] __main__: Hello world!, args=(None, <cimpl.Message object>), kwargs={}


%4|1671618714.562|CONFWARN|rdkafka#producer-13| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1671618714.562|CONFWARN|rdkafka#producer-13| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
%4|1671618714.568|CONFWARN|rdkafka#producer-14| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1671618714.568|CONFWARN|rdkafka#producer-14| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


[INFO] __main__: Goodbye world!, args=(None, <cimpl.Message object>), kwargs={}
[INFO] __main__: Hello world!, args=(None, <cimpl.Message object>), kwargs={}
[INFO] __main__: Goodbye world!, args=(None, <cimpl.Message object>), kwargs={}
[INFO] __main__: Hello world!, args=(None, <cimpl.Message object>), kwargs={}
[INFO] __main__: Goodbye world!, args=(None, <cimpl.Message object>), kwargs={}
[INFO] __main__: Hello world!, args=(None, <cimpl.Message object>), kwargs={}
[INFO] __main__: Goodbye world!, args=(None, <cimpl.Message object>), kwargs={}
[INFO] __main__: Hello world!, args=(None, <cimpl.Message object>), kwargs={}
[INFO] __main__: Goodbye world!, args=(None, <cimpl.Message object>), kwargs={}


[
│   <cimpl.Message object at 0x7f9ba7bfb1c0>,
│   <cimpl.Message object at 0x7f9ba7bfb0c0>,
│   <cimpl.Message object at 0x7f9ba7bfb6c0>,
│   <cimpl.Message object at 0x7f9ba7bfbac0>,
│   <cimpl.Message object at 0x7f9ba7bfbb40>
]

%5|1671619015.596|PARTCNT|rdkafka#producer-14| [thrd:main]: Topic test_topic_aioproducer_wqnhibbaev partition count changed from 3 to 1
